# 1. Edit and test the AI Logic

<div class="alert alert-info">
<b>Note:</b> this tutorial is run entirely on this Jupyter Notebook.
</div>

In this tutorial we understand how to edit the AI logic and test the changes using DevCSI.

## Preconditions
You have already done `2. Create & Run Applications` and have access to the boilerplate folder where the UI, Service and Skills are.

Navigate to the `<your-application-folder>/skills/qa.py` to find a code that is similar to the following snippet.

In [ ]:
from pharia_skill import ChatParams, Csi, IndexPath, Message, skill
from pydantic import BaseModel

NAMESPACE = "Studio"
COLLECTION = "papers"
INDEX = "asym-64"


class Input(BaseModel):
    question: str
    namespace: str = NAMESPACE
    collection: str = COLLECTION
    index: str = INDEX


class Output(BaseModel):
    answer: str | None


#@skill
def custom_rag(csi: Csi, input: Input) -> Output:
    index = IndexPath(
        namespace=input.namespace,
        collection=input.collection,
        index=input.index,
    )

    if not (documents := csi.search(index, input.question, 3, 0.5)):
        return Output(answer=None)

    context = "\n".join([d.content for d in documents])
    content = f"""Using the provided context documents below, answer the following question accurately and comprehensively. If the information is directly available in the context documents, cite it clearly. If not, use your knowledge to fill in the gaps while ensuring that the response is consistent with the given information. Do not fabricate facts or make assumptions beyond what the context or your knowledge base provides. Ensure that the response is structured, concise, and tailored to the specific question being asked.

Input: {context}

Question: {input.question}
"""
    message = Message.user(content)
    params = ChatParams(max_tokens=512)
    response = csi.chat("llama-3.1-8b-instruct", [message], params)
    return Output(answer=response.message.content)

Let's do a simple but effective change by improving the prompt and giving as output not only the response but also the sources.

In [ ]:
class Output(BaseModel):
    answer: str | None
    sources: list[str] | None

#@skill
def custom_rag(csi: Csi, input: Input) -> Output:
    index = IndexPath(
        namespace=input.namespace,
        collection=input.collection,
        index=input.index,
    )

    if not (documents := csi.search(index, input.question, 3, 0.5)):
        return Output(answer=None)

    context = "\n".join([d.content for d in documents])
    content = f"""Using the provided context documents below, answer the following question.
Format your response with the following sections: 
1. SUMMARY: A brief 1-2 sentence answer to the question 
2. DETAILS: A comprehensive explanation with specific information from the context 
3. SOURCES: References to the specific parts of the context you used, if applicable If the information is not available in the context documents, clearly state this and provide a general response based on your knowledge, marked as [GENERAL KNOWLEDGE].

Input: {context}

Question: {input.question}"""

    message = Message.user(content)
    params = ChatParams(max_tokens=512)
    response = csi.chat("llama-3.1-8b-instruct", [message], params)
    sources = [d.document_path.name for d in documents]
    return Output(answer=response.message.content, sources=sources)

We can now test the new skill code to check if the result is improved by using the DevCSI.

In [ ]:
from pharia_skill.testing import DevCsi

test_input = Input(question="What is an encoder?")

csi = DevCsi()
custom_rag(csi, test_input)

You can experiment with different variations of the prompt and different output types until you find the one that is more suitable for your application. You can also try with different input questions.

Let's move now to the skill publishing and deployment 
